In [2]:
%load_ext autoreload
%autoreload 2

from binance.client import Client
import os
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

from datetime import datetime
from libs import *
from utils import *

with open("C:\\Users\\thang\\OneDrive\\Documents\\binance_cres\\api_key.txt") as api_file:
    api_key = api_file.readline()
with open("C:\\Users\\thang\\OneDrive\\Documents\\binance_cres\\sec_key.txt") as api_file:
    api_secret = api_file.readline()
# make binance object
client = Client(api_key, api_secret)

# Parameters
target_coin = 'ETCUSDT'


In [3]:
## Raw data
# agg_trades = client.aggregate_trade_iter(symbol='ETCUSDT', start_str='60 minutes ago UTC')

# # iterate over the trade iterator
# df_dict = {'a':[], 'p':[], 'q':[], 'f':[], 'l':[],'T':[], 'm':[],'M':[]}
# for trade in agg_trades:
#     # prices.append(trade["p"])
#     df_dict['a'].append(trade["a"])
#     df_dict['p'].append(trade["p"])
#     df_dict['q'].append(trade["q"])
#     df_dict['f'].append(trade["f"])
#     df_dict['l'].append(trade["l"])
#     df_dict['T'].append(trade["T"])
#     df_dict['m'].append(trade["m"])
#     df_dict['M'].append(trade["M"])
#     print(trade)

# df = pd.DataFrame.from_dict(df_dict)
# interested_df = df[['p','q','T']]  # get price as p, quantity as q, Time as T

In [4]:
avg_data = client.get_klines(symbol=target_coin, interval=Client.KLINE_INTERVAL_30MINUTE)
cols = ['open_time',  # Open time
            'open',  # Open
            'high',  # High
            'low',  # Low
            'close',  # Close
            'vol',  # Volume
            'close_time',  # Close time
            'quote_ass_vol',  # Quote asset volume
            'no_trade',  # Number of trades
            'base_ass_vol',  # Taker buy base asset volume
            'qoute_ass_vol',  # Taker buy quote asset volume
            'ignore']   # Can be ignored
df = pd.DataFrame(avg_data, 
    columns = cols)

important_features_df = df[['close','close_time','vol']]  # get important features
important_features_df.close_time = important_features_df.close_time.apply(lambda x: x/1000)  # convert milisecond to second
important_features_df = data_converter(important_features_df)  # conver time data to sequence vector

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python37_64\lib\site-packages\pandas\core\generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
c:\Users\thang\Documents\pred_coin_price\ts_pred_viz\utils.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Day sin'] = np.sin(date_time * (2 * np.pi / day))
c:\Users\thang\Documents\pred_coin_price\ts_pred_viz\utils.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 

In [5]:
important_features_df.describe()

,close,vol,Day sin,Day cos,Year sin,Year cos
count,500.000000,500.000000,5.000000e+02,5.000000e+02,500.000000,500.000000
mean,67.835704,137950.289270,2.220755e-02,-1.947549e-02,0.560282,-0.826687
std,8.080250,122763.196009,7.078198e-01,7.071923e-01,0.042835,0.029044
min,42.432000,8626.440000,-1.000000e+00,-1.000000e+00,0.484869,-0.874587
25%,64.444000,56875.791250,-7.071067e-01,-7.071068e-01,0.523473,-0.852042
50%,67.748000,97151.412000,7.272603e-08,-7.272430e-08,0.561032,-0.827794
75%,74.585500,171530.258000,7.071068e-01,7.071067e-01,0.597469,-0.801892
max,82.197000,915454.300000,1.000000e+00,1.000000e+00,0.632712,-0.774387


In [6]:
# data splitting
# column_indices = {name: i for i, name in enumerate(important_features_df.columns)}

n = len(important_features_df)
train_df = important_features_df[0:int(n*0.7)]  # 70% train data
val_df = important_features_df[int(n*0.7):int(n*0.9)]  # 20% val data
test_df = important_features_df[int(n*0.9):]  # 10% test data

num_features = important_features_df.shape[1]  # number of features

In [7]:
# normalization
train_mean = train_df.mean()
train_std = train_df.std()

# normalize base on train_data distribution
train_df = (train_df - train_mean) / train_std
val_df = (val_df - train_mean) / train_std
test_df = (test_df - train_mean) / train_std


In [8]:
time_predictors = 24  # number of historical data as predictor
time_shift = 24  # number of timestamp to target
label_width = 1  # number of predicting timestamp

In [9]:
# def input_windows_maker(data_matrix):
#     inputs = data_matrix[:, :24, :]
#     return inputs

# def output_windows_maker(data_matrix):
#     inputs = data_matrix[:, 24:, :]
#     return inputs
# ds_output = ds.map(output_windows_maker)
# for k in ds_output:
#     print(k.numpy().shape)

In [10]:
# prepare data
single_step_window = WindowGenerator(
    input_width=time_predictors, label_width=label_width, shift=time_shift, train_df=train_df, val_df=val_df, test_df=test_df, label_columns=['close'])
single_step_window

Total window size: 48
Input indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
Label indices: [47]
Label column name(s): ['close']

In [11]:
count = 0
for element in single_step_window.train[0]:
    count += 1
    print(count)
    print(element.numpy())
    if len(element.numpy()) < 32:
        break
    if count == 1:
        train_input = element.numpy()
    else:
        train_input = np.append(train_input, element.numpy(), axis=0)

2947313e-01 -4.6015212e-01]
  [ 6.4674383e-01 -8.6104244e-01 -1.4479913e+00  1.5867855e-02
   -4.3947282e-01 -4.6981344e-01]
  [ 6.6573018e-01 -8.5704416e-01 -1.4359971e+00  2.0178506e-01
   -4.4947499e-01 -4.7946972e-01]]

 [[ 5.8988911e-01  8.3466512e-01  9.4533896e-01  1.0230476e+00
    8.0049467e-01  7.8641087e-01]
  [ 4.9297521e-01  2.2137575e+00  1.0662550e+00  8.8296771e-01
    7.9081559e-01  7.7613539e-01]
  [ 6.5696728e-01  6.6279739e-01  1.1681397e+00  7.2805154e-01
    7.8113389e-01  7.6586479e-01]
  ...
  [ 1.1748149e+00  2.3635588e+00 -5.8252537e-01 -1.3000757e+00
    5.9668088e-01  5.7164294e-01]
  [ 9.1609973e-01  2.0526426e+00 -7.4700093e-01 -1.2176701e+00
    5.8694655e-01  5.6146926e-01]
  [ 1.0323129e+00  9.5657992e-01 -8.9948231e-01 -1.1141584e+00
    5.7720965e-01  5.5130041e-01]]

 [[-2.3768653e-01 -1.7167050e-01  1.3698444e-01 -1.3963134e+00
   -7.9101825e-01 -8.0480450e-01]
  [-3.7758049e-01  4.1336939e-01 -4.6010718e-02 -1.4084990e+00
   -8.0110657e-01 -8.14285

In [12]:
train_input.shape

(288, 24, 6)

In [13]:
count = 0
for element in single_step_window.train[1]:
    count += 1
    print(count)
    print(element.numpy().shape)
    if len(element.numpy()) < 32:
        break
    if count == 1:
        train_label = element.numpy()
    else:
        train_label = np.append(train_label, element.numpy(), axis=0)

1
(32, 1, 1)
2
(32, 1, 1)
3
(32, 1, 1)
4
(32, 1, 1)
5
(32, 1, 1)
6
(32, 1, 1)
7
(32, 1, 1)
8
(32, 1, 1)
9
(32, 1, 1)
10
(15, 1, 1)


In [14]:
train_label.shape

(288, 1, 1)

In [15]:
### main model ###
# linear module
linear_input = tf.keras.Input(shape=(single_step_window.example[0].shape[1:]), batch_size=single_step_window.example[0].shape[0])  # batch 32 input shape = 24,6
x = tf.keras.layers.Flatten()(linear_input)
x = tf.keras.layers.Dense(units=128, activation='relu')(x)
x = tf.keras.layers.Dense(units=64, activation='relu')(x)
x = tf.keras.layers.Dense(units=1, activation='relu')(x)
linear_model = tf.keras.Model(inputs=linear_input, outputs=x)  # output = 32,1

# lstm module
lstm_input = tf.keras.Input(shape=(single_step_window.example[0].shape[1:]), batch_size=single_step_window.example[0].shape[0])
y = tf.keras.layers.LSTM(32, return_sequences=False)(lstm_input)
y = tf.keras.layers.Dense(units=1, activation='relu')(y)
lstm_model = tf.keras.Model(inputs=lstm_input, outputs=y)  # output = 32,1

# intersection
z = tf.keras.layers.concatenate([linear_model.output, lstm_model.output])
z = tf.keras.layers.Dense(units=64, activation='relu')(z)
z = tf.keras.layers.Dense(units=1, activation='relu')(z)
comb_model = tf.keras.Model(inputs=[linear_input, lstm_input], outputs=[z])


# training setting

MAX_EPOCHS = 20

def compile_and_fit(model, window, patience=2):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(),
                metrics=[tf.metrics.MeanAbsoluteError()])

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

In [ ]:
# tf.keras.utils.plot_model(comb_model, "multi_input_and_output_model.png", show_shapes=True)

In [16]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                patience=2,
                                                mode='min')
comb_model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(),
                metrics=[tf.metrics.MeanAbsoluteError()])
comb_model.summary()


# define val data

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(32, 24, 6)]        0                                            
__________________________________________________________________________________________________
flatten (Flatten)               (32, 144)            0           input_1[0][0]                    
__________________________________________________________________________________________________
dense (Dense)                   (32, 128)            18560       flatten[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            [(32, 24, 6)]        0                                            
_______________________________________________________________________________________

In [17]:
# training
comb_model.fit([train_input, train_input], [train_label], epochs=MAX_EPOCHS )

Epoch 1/20
9/9 [==============================] - 0s 4ms/step - loss: 0.6349 - mean_absolute_error: 0.6867
Epoch 2/20
9/9 [==============================] - 0s 4ms/step - loss: 0.6120 - mean_absolute_error: 0.6681
Epoch 3/20
9/9 [==============================] - 0s 4ms/step - loss: 0.5610 - mean_absolute_error: 0.6374
Epoch 4/20
9/9 [==============================] - 0s 4ms/step - loss: 0.5466 - mean_absolute_error: 0.6187
Epoch 5/20
9/9 [==============================] - 0s 4ms/step - loss: 0.5298 - mean_absolute_error: 0.6020
Epoch 6/20
9/9 [==============================] - 0s 4ms/step - loss: 0.5209 - mean_absolute_error: 0.5997
Epoch 7/20
9/9 [==============================] - 0s 3ms/step - loss: 0.5064 - mean_absolute_error: 0.5965
Epoch 8/20
9/9 [==============================] - 0s 3ms/step - loss: 0.4993 - mean_absolute_error: 0.5812
Epoch 9/20
9/9 [==============================] - 0s 3ms/step - loss: 0.4882 - mean_absolute_error: 0.5763
Epoch 10/20
9/9 [====================

In [ ]:
# evaluation
val_performance = {}
performance = {}
val_performance['multi_modal'] = comb_model.evaluate([train_input, train_input], [train_label])
performance['multi_modal'] = comb_model.evaluate([train_input, train_input], [train_label])

In [ ]:
val_performance

In [ ]:
plt.bar(x = range(len(train_df.columns)),
        height=lstm_model.layers[0].kernel[:,0].numpy())
axis = plt.gca()
axis.set_xticks(range(len(train_df.columns)))
_ = axis.set_xticklabels(train_df.columns, rotation=90)

In [ ]:
print('Input shape:', wide_window.example[0].shape)
print('Output shape:', linear(wide_window.example[0]).shape)